<a href="https://colab.research.google.com/github/AnshuMishra01/LLM-Scratch/blob/main/Bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
#Hyperparameters
block_size = 8
batch_size = 4
max_iters = 1000000
learning_rate = 3e-3
eval_iters = 250
dropout = 0.2

In [ ]:
with open('prince_of_lover.txt', 'r', encoding='utf-8') as f:
  text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [ ]:
vocab_size

93

In [ ]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l:"".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([92, 45, 62, 59,  1, 41, 72, 69, 64, 59, 57, 74,  1, 32, 75, 74, 59, 68,
        56, 59, 72, 61,  1, 59, 27, 69, 69, 65,  1, 69, 60,  1, 26,  1, 70, 72,
        63, 68, 57, 59,  1, 69, 60,  1, 66, 69, 76, 59, 72, 73,  0,  1,  1,  1,
         1,  0, 45, 62, 63, 73,  1, 59, 56, 69, 69, 65,  1, 63, 73,  1, 60, 69,
        72,  1, 74, 62, 59,  1, 75, 73, 59,  1, 69, 60,  1, 55, 68, 79, 69, 68,
        59,  1, 55, 68, 79, 77, 62, 59, 72, 59])


Q) Why here we are making tensors ?

In [ ]:
test = pd.DataFrame(encode(text));
test.shape

(573706, 1)

In [ ]:
data.shape

torch.Size([573706])

In [ ]:
n = int(0.8*len(data))
train = data[:n]
val = data[n:]

In [ ]:
# block_size = 5

# x= train[:block_size]
# y = train[1:block_size+1]
# for t in range(block_size):
#   context = x[:t+1]
#   target = y[t]
#   print("when input is ", context, "Output is ", target)

In [ ]:
def get_batch(split):
  data = train if split =='train' else val
  ix = torch.randint(len(data) - block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x,y

x,y = get_batch('train')
print('inputs : ')
print(x)
print('outputs : ')
print(y)

inputs : 
tensor([[67, 69, 68, 63, 68, 61,  0, 62],
        [68, 69,  0, 73, 63, 61, 68, 73],
        [66, 79,  1, 62, 63, 68, 74,  1],
        [58,  9,  1, 55,  1, 58, 55, 68]], device='cuda:0')
outputs : 
tensor([[69, 68, 63, 68, 61,  0, 62, 59],
        [69,  0, 73, 63, 61, 68, 73,  1],
        [79,  1, 62, 63, 68, 74,  1, 63],
        [ 9,  1, 55,  1, 58, 55, 68, 61]], device='cuda:0')


## Bigram Langauge Model

In [ ]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)

    if targets is None:
      loss = None
    else:
      B, T, C  = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits ,loss

  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self.forward(index, targets=None)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim =-1)
      index_next = torch.multinomial(probs, num_samples=1)
      index = torch.cat((index, index_next), dim=1)
    return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device) ##INT64
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


iVPy,4‘Oc;H7y—qNbLLF2QIfqMzT8äx*EWX64ïd1zx$.?4*5﻿5e!Nr0’;‘UDw2;rc﻿][”(/Y‘m5E•XEIEpï4﻿!?sFWI’o’Ot0QAFOZw9Z-q﻿TaEHV?Ea
Q•r;1H**UEQN2“BVGP/’2r-#k,V ,VmbY /Ws?Ci.ar(“8äh:e•™d7jsJjmvD“uGV?e•TJsJ0hwœ15Iwj2cHV6$]pJhMœè)kwcPXè8hV6pïjN﻿TF﻿.fIaT™LgN2(h:C,4jnA/’?omA9VJ1X—;3 ZbY,!2bF﻿J.Wl i-%;EJSB9ï,7h”cWGFWn%a
—èL—f™hZ—SUQwbOp9ooRCQ‘i
np”qm?4AF53t™4H6-A”p4:iOKiQMZ ,7pe•.Yg“kH1:4_sEW tT!lb﻿TEqjNY[ZQ7ybX-uQœT8’8•

UWvs2Ajq?,x—Lgwo$T]äHC#nGïP“u-wb ZDwWP$qfE:egqtIZDg$WykZ?KZkw/D6$NS‘”l ipA”Bï1P™—sT’!o6$E.hFd﻿•


In [ ]:
#create a PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  xb, yb = get_batch('train')
  logits, loss = model.forward(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())

2.275867223739624


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


“IShepe oino thofo t

Mibem. thon and alsweded shir, ider
llderer htithad, Che

“Wh, ine wetrowathat, hed ined.

athunge d. aghe
sad,”
ort antinamo n athand a’sthenet berkn hiof akenghay “Ifis,
hald indad.

Chofou lenghe, bolecess renteghas rse
cinathok d, tiot wate As vinaisss arond ours qurerals
“I ghew iofowa harin. neman passt s ase tlitinng larsy lorn ar in. te Dos ilioony. ind alor I tat ther hery irone d, ry, was, ppeno Thead so s me t ceadrit s

“He Haise f win. whetiffr has
m.”
I tifoor


In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] =loss.item()
    out[split] = losses.mean()
  model.train()
  return out